# EASY

In [ ]:
from typing import List, Tuple
import random

class Matrix:
    def __init__(self, n: int, m: int):
        self.n = n  # Число строк
        self.m = m  # Число столбцов
        self.rows = []  # Список строк в разреженном формате

        # Инициализация пустых строк
        for _ in range(n):
            self.rows.append([0] * m)  # Строки будут обычными списками с нулями

    def add_value(self, row: int, col: int, value: float):
        """
        Добавляет значение в матрицу по индексу (row, col).
        """
        self.rows[row - 1][col - 1] = value  # Индексация с 1, но для списка используем row-1 и col-1

    def get_element(self, row: int, col: int) -> float:
        """
        Возвращает значение элемента матрицы.
        """
        return self.rows[row - 1][col - 1]  # Индексация с 1, но для списка используем row-1 и col-1

    def print_matrix(self):
        for row in self.rows:
            print(row)


In [85]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    n = A.n
    # Преобразуем разреженные матрицы в плотный формат
    aug = [[A.get_element(i + 1, j + 1) for j in range(n)] + [b.get_element(i + 1, 1)] for i in range(n)]

    for i in range(n):
        # Поиск главного элемента
        max_row = max(range(i, n), key=lambda r: abs(aug[r][i]))
        if aug[max_row][i] == 0:
            raise ValueError("Система несовместна или имеет бесконечно много решений")

        # Перестановка строк
        aug[i], aug[max_row] = aug[max_row], aug[i]

        # Прямой ход
        for j in range(i + 1, n):
            factor = aug[j][i] / aug[i][i]
            for k in range(i, n + 1):
                aug[j][k] -= factor * aug[i][k]

    # Обратный ход
    x = [0] * n
    for i in reversed(range(n)):
        s = sum(aug[i][j] * x[j] for j in range(i + 1, n))
        if aug[i][i] == 0:
            raise ValueError("Система несовместна или имеет бесконечно много решений")
        x[i] = (aug[i][n] - s) / aug[i][i]

    # Преобразуем решение в список матриц (каждая переменная как матрица 1x1)
    result = []
    for val in x:
        m = Matrix(1, 1)
        m.add_value(1, 1, val)
        result.append(m)

    return result

In [87]:
def center_data(X: 'Matrix') -> 'Matrix':
    n, m = X.n, X.m
    means = [0.0] * m

    # Вычисляем средние значения по столбцам
    for j in range(m):
        col_sum = sum(X.get_element(i + 1, j + 1) for i in range(n))
        means[j] = col_sum / n

    # Центрируем
    X_centered = Matrix(n, m)
    for i in range(n):
        for j in range(m):
            val = X.get_element(i + 1, j + 1) - means[j]
            X_centered.add_value(i + 1, j + 1, val)

    return X_centered


def covariance_matrix(X_centered: 'Matrix') -> 'Matrix':
    n, m = X_centered.n, X_centered.m
    C = Matrix(m, m)

    for i in range(m):
        for j in range(m):
            cov = sum(
                X_centered.get_element(k + 1, i + 1) * X_centered.get_element(k + 1, j + 1)
                for k in range(n)
            ) / (n - 1)
            C.add_value(i + 1, j + 1, cov)

    return C

In [91]:
def main():
    # Ввод данных для решения СЛАУ методом Гаусса
    print("=== Ввод данных для метода Гаусса ===")
    n = int(input("Введите размерность матрицы A (n): "))

    # Ввод матрицы коэффициентов A
    A = Matrix(n, n)
    print(f"Введите элементы матрицы A размером {n}x{n} (по строкам):")
    for i in range(n):
        row_values = input(f"Строка {i+1}: ").split()
        for j in range(n):
            A.add_value(i + 1, j + 1, float(row_values[j]))

    # Ввод вектора правых частей b
    b = Matrix(n, 1)
    print(f"Введите элементы вектора правых частей b размером {n}:")
    for i in range(n):
        value = float(input(f"Элемент {i+1}: "))
        b.add_value(i + 1, 1, value)

    # Решение СЛАУ методом Гаусса
    solution = gauss_solver(A, b)
    print("Решение системы:")
    for i, val in enumerate(solution):
        print(f"x{i+1} =", val.get_element(1, 1))

    n = int(input("Введите количество строк матрицы X (n): "))
    m = int(input("Введите количество столбцов матрицы X (m): "))

    X = Matrix(n, m)
    print(f"Введите элементы матрицы X размером {n}x{m} (по строкам):")
    for i in range(n):
        row_values = input(f"Строка {i+1}: ").split()
        for j in range(m):
            X.add_value(i + 1, j + 1, float(row_values[j]))

    # Центрирование данных
    Xc = center_data(X)
    print("Центрированные данные:")
    for i in range(Xc.n):
        row = [Xc.get_element(i + 1, j + 1) for j in range(Xc.m)]
        print(row)

    # Вычисление ковариационной матрицы
    Cov = covariance_matrix(Xc)
    print("\nКовариационная матрица:")
    for i in range(Cov.n):
        row = [Cov.get_element(i + 1, j + 1) for j in range(Cov.m)]
        print(row)
if __name__ == "__main__":
    main()

=== Ввод данных для метода Гаусса ===
Введите размерность матрицы A (n): 2
Введите элементы матрицы A размером 2x2 (по строкам):
Строка 1: 1 2
Строка 2: 2 1
Введите элементы вектора правых частей b размером 2:
Элемент 1: 2
Элемент 2: 1
Решение системы:
x1 = 0.0
x2 = 1.0
Введите количество строк матрицы X (n): 2
Введите количество столбцов матрицы X (m): 2
Введите элементы матрицы X размером 2x2 (по строкам):
Строка 1: 1 2
Строка 2: 2 1
Центрированные данные:
[-0.5, 0.5]
[0.5, -0.5]

Ковариационная матрица:
[0.5, -0.5]
[-0.5, 0.5]


# NORMAL & HARD

In [92]:
def orthogonal_vectors(v: list) -> list:
    """Генерация ортогональных векторов."""
    dim = len(v)
    orthogonals = []

    # Первый ортогональный вектор
    if dim >= 2:
        orth1 = [0.0] * dim
        orth1[0] = 1.0
        if abs(v[-1]) > 1e-10:  # Проверка на малое значение
            orth1[-1] = -v[0] / v[-1]
        orthogonals.append(orth1)

    # Второй ортогональный вектор
    if dim >= 3:
        orth2 = [0.0] * dim
        orth2[1] = 1.0
        if abs(v[-1]) > 1e-10:  # Проверка на малое значение
            orth2[-1] = -(v[1] / v[-1])
        orthogonals.append(orth2)

    return orthogonals


def normalize_vector(v: list) -> list:
    """Нормализация вектора."""
    norm = sum(x ** 2 for x in v) ** 0.5
    return [x / norm for x in v]


def normalize_signs(eigenvectors: list) -> list:
    """Нормализация знаков собственных векторов."""
    normalized_vectors = []
    for vector in eigenvectors:
        # Находим первый ненулевой элемент
        for i, value in enumerate(vector):
            if abs(value) > 1e-10:  # Проверяем, что элемент не равен нулю
                sign = 1 if value > 0 else -1
                break
        # Применяем знак ко всему вектору
        normalized_vector = [x * sign for x in vector]
        normalized_vectors.append(normalized_vector)
    return normalized_vectors


def eigen(matrix: Matrix, tol=1e-10, max_iter=1000) -> tuple:
    def dot_product(v1, v2):
        return sum(x * y for x, y in zip(v1, v2))

    def vector_norm(v):
        return sum(x ** 2 for x in v) ** 0.5

    def matrix_vector_multiply(mat, vec):
        return [dot_product([mat.get_element(i + 1, j + 1) for j in range(mat.m)], vec) for i in range(mat.n)]

    def orthogonalize(v_new, eigenvectors_found):
        """Ортогонализация нового вектора относительно уже найденных собственных векторов."""
        for v_found in eigenvectors_found:
            projection = dot_product(v_new, v_found)
            v_new = [v_new[i] - projection * v_found[i] for i in range(len(v_new))]

        # Проверка нормы вектора
        norm = vector_norm(v_new)
        if norm < 1e-10:
            raise ValueError("Начальный вектор стал слишком маленьким после ортогонализации.")

        return [x / norm for x in v_new]

    eigenvalues = []
    eigenvectors = []

    mat = Matrix(matrix.n, matrix.m)
    for i in range(matrix.n):
        for j in range(matrix.m):
            mat.add_value(i + 1, j + 1, matrix.get_element(i + 1, j + 1))  # Копия текущей матрицы

    for _ in range(matrix.m):
        # Начальный вектор (фиксированный)
        b_k = [1.0] * matrix.m  # Вектор из единиц

        # Ортогонализация начального вектора относительно уже найденных собственных векторов
        b_k = orthogonalize(b_k, eigenvectors)

        for iteration in range(max_iter):
            b_k1 = matrix_vector_multiply(mat, b_k)
            norm = vector_norm(b_k1)

            # Проверка на нулевую норму
            if norm < tol:
                break

            b_k1 = [x / norm for x in b_k1]

            if all(abs(b_k[i] - b_k1[i]) < tol for i in range(matrix.m)):
                break
            b_k = b_k1

        eigenvalue = dot_product(matrix_vector_multiply(mat, b_k), b_k) / dot_product(b_k, b_k)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(b_k)

        # Обнуление влияния текущего собственного вектора
        outer_product = [[b_k[i] * b_k[j] for j in range(matrix.m)] for i in range(matrix.m)]
        for i in range(matrix.m):
            for j in range(matrix.m):
                mat.add_value(i + 1, j + 1, mat.get_element(i + 1, j + 1) - eigenvalue * outer_product[i][j])

        # Если матрица стала нулевой, завершаем поиск
        if all(abs(mat.get_element(i + 1, j + 1)) < tol for i in range(matrix.n) for j in range(matrix.m)):
            break

    # Если остались нулевые собственные значения, добавляем ортогональные векторы
    while len(eigenvectors) < matrix.m:
        orthogonals = orthogonal_vectors(eigenvectors[0])
        for orth in orthogonals:
            eigenvectors.append(normalize_vector(orth))  # Нормализуем ортогональный вектор
            eigenvalues.append(0.0)
            if len(eigenvectors) == matrix.m:
                break

    # Нормализация знаков собственных векторов
    eigenvectors = normalize_signs(eigenvectors)

    return eigenvalues, eigenvectors


def sort_eigen(eigenvalues: list, eigenvectors: list) -> tuple:
    """Сортировка собственных значений и векторов по убыванию собственных значений."""
    paired = sorted(zip(eigenvalues, eigenvectors), key=lambda pair: pair[0], reverse=True)
    eigenvalues, eigenvectors = zip(*paired)
    return list(eigenvalues), list(eigenvectors)


def project_data(X: Matrix, eigenvectors: list, k: int) -> Matrix:
    """Проекция данных на первые k главных компонент."""
    # Построение матрицы W_k (m x k), где каждый столбец — собственный вектор
    W_k = [[eigenvectors[i][j] for i in range(k)] for j in range(X.m)]

    for i in W_k:
      print(i)

    # Создание матрицы проекции
    X_proj = Matrix(X.n, k)
    for i in range(X.n):  # По строкам X
        for l in range(k):  # По столбцам W_k
            # Умножение строки X на столбец W_k
            val = sum(X.get_element(i + 1, j + 1) * W_k[j][l] for j in range(X.m))
            X_proj.add_value(i + 1, l + 1, val)

    return X_proj

In [ ]:
def explained_variance_ratio(eigenvalues: list, k: int) -> float:
    """Вычисление доли объяснённой дисперсии."""
    total_variance = sum(eigenvalues)
    explained_variance = sum(eigenvalues[:k])
    return explained_variance / total_variance

def pca(X: Matrix, k: int) -> tuple:
    """
    Реализация алгоритма PCA.

    Вход:
        X: матрица данных (n x m)
        k: число главных компонент

    Выход:
        X_proj: проекция данных (n x k)
        explained_variance_ratio: доля объяснённой дисперсии
    """

    Y = center_data(X)
    print("Центрированная матрица")
    for i in range(Y.n):
        row = [Y.get_element(i + 1, j + 1) for j in range(Y.m)]
        print(row)

    print("\n")

    # Шаг 2: Вычисление матрицы ковариаций
    cov_matrix = covariance_matrix(Y)
    print("Ковариционная матрица")
    for i in range(cov_matrix.n):
        row = [cov_matrix.get_element(i + 1, j + 1) for j in range(cov_matrix.m)]
        print(row)

    print("\n")

    # Шаг 3: Нахождение собственных значений и векторов
    eigenvalues, eigenvectors = eigen(cov_matrix)
    print("Собственные значения")
    print(eigenvalues)
    print("Собственные вектора")
    print(eigenvectors)

    # Сортировка собственных значений и векторов
    eigenvalues, eigenvectors = sort_eigen(eigenvalues, eigenvectors)

    # Шаг 4: Проекция данных на главные компоненты
    X_proj = project_data(Y, eigenvectors, k)

    # Доля объяснённой дисперсии
    explained_variance_ratio_value = explained_variance_ratio(eigenvalues, k)

    return X_proj, explained_variance_ratio_value

In [ ]:
# Создание матрицы данных
X = Matrix(3, 3)
X.add_value(1, 1, 1.0)
X.add_value(1, 2, 2.0)
X.add_value(1, 3, 3.0)
X.add_value(2, 1, 4.0)
X.add_value(2, 2, 5.0)
X.add_value(2, 3, 6.0)
X.add_value(3, 1, 7.0)
X.add_value(3, 2, 8.0)
X.add_value(3, 3, 9.0)

# Применение PCA
X_proj, explained_variance_ratio_value = pca(X, k=2)

print("Проекция данных:")
X_proj.print_matrix()
print("Доля объяснённой дисперсии:", explained_variance_ratio_value)

Центрированная матрица
[-3.0, -3.0, -3.0]
[0.0, 0.0, 0.0]
[3.0, 3.0, 3.0]


Ковариционная матрица
[9.0, 9.0, 9.0]
[9.0, 9.0, 9.0]
[9.0, 9.0, 9.0]


Собственные значения
[27.000000000000004, 0.0, 0.0]
Собственные вектора
[[0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [0.7071067811865475, 0.0, -0.7071067811865475], [0.0, 0.7071067811865475, -0.7071067811865475]]
[0.5773502691896258, 0.7071067811865475]
[0.5773502691896258, 0.0]
[0.5773502691896258, -0.7071067811865475]
Проекция данных:
[-5.196152422706633, 0.0]
[0.0, 0.0]
[5.196152422706633, 0.0]
Доля объяснённой дисперсии: 1.0


In [ ]:
X = Matrix(5, 5)
X.add_value(1, 1, 1.0)
X.add_value(1, 2, 2.0)
X.add_value(1, 3, 3.0)
X.add_value(1, 4, 4.0)
X.add_value(1, 5, 5.0)

X.add_value(2, 1, 6.0)
X.add_value(2, 2, 7.0)
X.add_value(2, 3, 8.0)
X.add_value(2, 4, 9.0)
X.add_value(2, 5, 10.0)

X.add_value(3, 1, 11.0)
X.add_value(3, 2, 12.0)
X.add_value(3, 3, 13.0)
X.add_value(3, 4, 14.0)
X.add_value(3, 5, 15.0)

X.add_value(4, 1, 16.0)
X.add_value(4, 2, 17.0)
X.add_value(4, 3, 18.0)
X.add_value(4, 4, 19.0)
X.add_value(4, 5, 20.0)

X.add_value(5, 1, 21.0)
X.add_value(5, 2, 22.0)
X.add_value(5, 3, 23.0)
X.add_value(5, 4, 24.0)
X.add_value(5, 5, 25.0)

# Применение PCA
X_proj, explained_variance_ratio_value = pca(X, k=2)

print("Проекция данных:")
X_proj.print_matrix()
print("Доля объяснённой дисперсии:", explained_variance_ratio_value)

Центрированная матрица
[-10.0, -10.0, -10.0, -10.0, -10.0]
[-5.0, -5.0, -5.0, -5.0, -5.0]
[0.0, 0.0, 0.0, 0.0, 0.0]
[5.0, 5.0, 5.0, 5.0, 5.0]
[10.0, 10.0, 10.0, 10.0, 10.0]


Ковариционная матрица
[62.5, 62.5, 62.5, 62.5, 62.5]
[62.5, 62.5, 62.5, 62.5, 62.5]
[62.5, 62.5, 62.5, 62.5, 62.5]
[62.5, 62.5, 62.5, 62.5, 62.5]
[62.5, 62.5, 62.5, 62.5, 62.5]


Собственные значения
[312.50000000000006, 0.0, 0.0, 0.0, 0.0]
Собственные вектора
[[0.4472135954999579, 0.4472135954999579, 0.4472135954999579, 0.4472135954999579, 0.4472135954999579], [0.7071067811865475, 0.0, 0.0, 0.0, -0.7071067811865475], [0.0, 0.7071067811865475, 0.0, 0.0, -0.7071067811865475], [0.7071067811865475, 0.0, 0.0, 0.0, -0.7071067811865475], [0.0, 0.7071067811865475, 0.0, 0.0, -0.7071067811865475]]
[0.4472135954999579, 0.7071067811865475]
[0.4472135954999579, 0.0]
[0.4472135954999579, 0.0]
[0.4472135954999579, 0.0]
[0.4472135954999579, -0.7071067811865475]
Проекция данных:
[-22.360679774997898, 0.0]
[-11.180339887498949, 0.

In [123]:
from matplotlib.figure import Figure
import matplotlib.pyplot as plt

def plot_pca_projection(X_proj: 'Matrix') -> Figure:
    """
    Визуализация проекции данных на первые две главные компоненты.

    Вход:
        X_proj: проекция данных (n x 2)

    Выход:
        объект Figure из Matplotlib
    """
    # Создание нового объекта Figure
    fig = Figure()
    ax = fig.add_subplot(111)

    # Получение данных для первой и второй компонент
    x = [X_proj.get_element(i + 1, 1) for i in range(X_proj.n)]  # Первая компонента (x-координата)
    y = [X_proj.get_element(i + 1, 2) for i in range(X_proj.n)]  # Вторая компонента (y-координата)

    # Построение точечного графика
    ax.scatter(x, y, color='blue', alpha=0.7, label="Данные")

    x_centre = (max(x) - min(x)) / 2
    y_centre = (max(y) - min(y)) / 2

    # Автоматический расчет пределов осей
    x_min, x_max = 0 - x_centre, x_centre
    y_min, y_max = 0 - y_centre, y_centre

    # Установка пределов осей
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # Перемещение осей в центр
    ax.spines['left'].set_position('zero')  # Левая граница (ось Y) в центре
    ax.spines['bottom'].set_position('zero')  # Нижняя граница (ось X) в центре

    # Удаление верхней и правой границ
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Добавление стрелок на концах осей
    ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)  # Стрелка на оси X
    ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)  # Стрелка на оси Y

    # Настройка графика
    ax.set_title("Проекция данных на первые две главные компоненты")
    ax.set_xlabel("Первая главная компонента (x)", labelpad=10, loc='right')  # Метка снизу
    ax.set_ylabel("Вторая главная компонента (y)", labelpad=10, loc='top')    # Метка слева
    ax.xaxis.set_label_coords(1.05, -0.05)  # Смещение метки оси X
    ax.yaxis.set_label_coords(-0.05, 1.05)  # Смещение метки оси Y

    # Добавление сетки
    ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax.legend()

    return fig


# X_proj.print_matrix()
# Визуализация проекции
fig = plot_pca_projection(X_proj)

# Отображение графика
plt.show(fig)

fig.savefig("pca_projection.png", dpi=300)

In [ ]:
def main():
    # Ввод данных для решения СЛАУ методом Гаусса
    n = int(input("Введите размерность матрицы A (n): "))

    # Ввод матрицы коэффициентов A
    A = Matrix(n, n)
    print(f"Введите элементы матрицы A размером {n}x{n} (по строкам):")
    for i in range(n):
        row_values = input(f"Строка {i+1}: ").split()
        for j in range(n):
            A.add_value(i + 1, j + 1, float(row_values[j]))

    # Применение PCA
    X_proj, explained_variance_ratio_value = pca(A, k=2)

    print("Проекция данных:")
    X_proj.print_matrix()
    print("Доля объяснённой дисперсии:", explained_variance_ratio_value)

    # X_proj.print_matrix()
    # Визуализация проекции
    fig = plot_pca_projection(X_proj)

    X_proj.print_matrix()
    # Отображение графика
    plt.show(fig)
    fig.savefig("test_x_proj.png", dpi=300)


if __name__ == "__main__":
    main()

Введите размерность матрицы A (n): 9
Введите элементы матрицы A размером 9x9 (по строкам):
Строка 1: 1 2 3 4 3 2 1 7 0
Строка 2: 0 9 8 4 3 4 2 1 2
Строка 3: 1 9 2 8 3 7 4 6 5
Строка 4: 9 8 7 6 5 4 3 2 1
Строка 5: 1 2 3 4 5 6 7 8 9
Строка 6: 10 11 12 13 14 15 16 17 18
Строка 7: 2 3 1 6 5 4 8 9 7
Строка 8: 0 0 0 0 0 0 0 0 0
Строка 9: 9 18 7 16 5 14 3 12 1
Центрированная матрица
[-2.6666666666666665, -4.888888888888889, -1.7777777777777777, -2.7777777777777777, -1.7777777777777777, -4.222222222222222, -3.8888888888888893, 0.11111111111111072, -4.777777777777778]
[-3.6666666666666665, 2.1111111111111107, 3.2222222222222223, -2.7777777777777777, -1.7777777777777777, -2.2222222222222223, -2.8888888888888893, -5.888888888888889, -2.7777777777777777]
[-2.6666666666666665, 2.1111111111111107, -2.7777777777777777, 1.2222222222222223, -1.7777777777777777, 0.7777777777777777, -0.8888888888888893, -0.8888888888888893, 0.22222222222222232]
[5.333333333333334, 1.1111111111111107, 2.2222222222222223, 

In [30]:
def test(X):
  A = Matrix(len(X), len(X))

  for i in range(len(X)):
    for j in range(len(X[i])):
      A.add_value(i + 1, j + 1, X[j][i])

  A.print_matrix()

  A_proj, explained_variance_ratio_value = pca(A, k=2)

  print("Проекция данных:")
  A_proj.print_matrix()
  print("Доля объяснённой дисперсии:", explained_variance_ratio_value)

  # X_proj.print_matrix()
  # Визуализация проекции
  fig = plot_pca_projection(A_proj)

  A_proj.print_matrix()
  # Отображение графика
  plt.show(fig)
  fig.savefig("test_a_proj.png", dpi=300)

  print("test is actually")

X = [[1, 2, 3, 4, 3, 2, 1, 7, 0],
      [0, 9, 8, 4, 3, 4, 2, 1, 2],
      [1, 9, 2, 8, 3, 7, 4, 6, 5],
      [9, 8, 7, 6, 5, 4, 3, 2, 1],
      [1, 2, 3, 4, 5, 6, 7, 8, 9],
      [10, 11, 12, 13, 14, 15, 16, 17, 18],
      [2, 3, 1, 6, 5, 4, 8, 9, 7],
      [0, 0, 0, 0, 0, 0, 0, 0, 0],
      [9, 18, 7, 16, 5, 14, 3, 12, 1]]

test(X)

[1, 0, 1, 9, 1, 10, 2, 0, 9]
[2, 9, 9, 8, 2, 11, 3, 0, 18]
[3, 8, 2, 7, 3, 12, 1, 0, 7]
[4, 4, 8, 6, 4, 13, 6, 0, 16]
[3, 3, 3, 5, 5, 14, 5, 0, 5]
[2, 4, 7, 4, 6, 15, 4, 0, 14]
[1, 2, 4, 3, 7, 16, 8, 0, 3]
[7, 1, 6, 2, 8, 17, 9, 0, 12]
[0, 2, 5, 1, 9, 18, 7, 0, 1]
Центрированная матрица
[-1.5555555555555554, -3.6666666666666665, -4.0, 4.0, -4.0, -4.0, -3.0, 0.0, -0.44444444444444464]
[-0.5555555555555554, 5.333333333333334, 4.0, 3.0, -3.0, -3.0, -2.0, 0.0, 8.555555555555555]
[0.44444444444444464, 4.333333333333334, -3.0, 2.0, -2.0, -2.0, -4.0, 0.0, -2.4444444444444446]
[1.4444444444444446, 0.3333333333333335, 3.0, 1.0, -1.0, -1.0, 1.0, 0.0, 6.555555555555555]
[0.44444444444444464, -0.6666666666666665, -2.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.444444444444445]
[-0.5555555555555554, 0.3333333333333335, 2.0, -1.0, 1.0, 1.0, -1.0, 0.0, 4.555555555555555]
[-1.5555555555555554, -1.6666666666666665, -1.0, -2.0, 2.0, 2.0, 3.0, 0.0, -6.444444444444445]
[4.444444444444445, -2.6666666666666665, 1.0, -3.0

In [ ]:
def reconstruction_error(X_orig: 'Matrix', X_recon: 'Matrix') -> float:
    """
    Вычисление среднеквадратической ошибки восстановления данных.

    Вход:
        X_orig: исходные данные (n x m)
        X_recon: восстановленные данные (n x m)

    Выход:
        среднеквадратическая ошибка MSE
    """
    # Проверка размерности матриц
    if X_orig.n != X_recon.n or X_orig.m != X_recon.m:
        raise ValueError("Размерности исходных и восстановленных данных не совпадают.")

    n, m = X_orig.n, X_orig.m  # Размеры матрицы
    total_elements = n * m

    # Вычисление суммы квадратов разностей
    mse_sum = 0.0
    for i in range(n):
        for j in range(m):
            diff = X_orig.get_element(i + 1, j + 1) - X_recon.get_element(i + 1, j + 1)
            mse_sum += diff ** 2

    # Вычисление среднеквадратической ошибки
    mse = mse_sum / total_elements

    return mse

In [ ]:
# Создание исходных данных
X_orig = Matrix(3, 3)
X_orig.add_value(1, 1, 1.0)
X_orig.add_value(1, 2, 2.0)
X_orig.add_value(1, 3, 3.0)
X_orig.add_value(2, 1, 4.0)
X_orig.add_value(2, 2, 5.0)
X_orig.add_value(2, 3, 6.0)
X_orig.add_value(3, 1, 7.0)
X_orig.add_value(3, 2, 8.0)
X_orig.add_value(3, 3, 9.0)

# Создание восстановленных данных
X_recon = Matrix(3, 3)
X_recon.add_value(1, 1, 1.1)
X_recon.add_value(1, 2, 2.1)
X_recon.add_value(1, 3, 3.1)
X_recon.add_value(2, 1, 4.1)
X_recon.add_value(2, 2, 5.1)
X_recon.add_value(2, 3, 6.1)
X_recon.add_value(3, 1, 7.1)
X_recon.add_value(3, 2, 8.1)
X_recon.add_value(3, 3, 9.1)

# Вычисление MSE
mse = reconstruction_error(X_orig, X_recon)
print(f"Среднеквадратическая ошибка восстановления: {mse}")

Среднеквадратическая ошибка восстановления: 0.009999999999999957


# EXPERT

In [31]:
def auto_select_k(eigenvalues: List[float], threshold: float = 0.95) -> int:
    """
    Вход:
        eigenvalues: список собственных значений
        threshold: порог объяснённой дисперсии
    Выход: оптимальное число главных компонент k
    """
    for k in range(1, len(eigenvalues) + 1):
        val = explained_variance_ratio(eigenvalues, k)
        if val > threshold:
            return k
    return len(eigenvalues)

In [83]:
import math

def handle_missing_values(X: Matrix) -> Matrix:
    """
    Вход: матрица данных X (n×m) с возможными NaN
    Выход: матрица данных X_filled (n×m) без NaN
    """
    n = len(X)
    m = len(X[0]) if n > 0 else 0

    # Вычисляем среднее по каждому столбцу (игнорируя NaN)
    column_means = []
    for j in range(m):
        total = 0.0
        count = 0
        for i in range(n):
            if not math.isnan(X[i][j]):
                total += X[i][j]
                count += 1
        mean = total / count if count > 0 else 0.0
        column_means.append(mean)

    # Создаём новую матрицу с заменой NaN на среднее по столбцу
    X_filled = []
    for i in range(n):
        row = []
        for j in range(m):
            if math.isnan(X[i][j]):
                row.append(column_means[j])
            else:
                row.append(X[i][j])
        X_filled.append(row)

    return X_filled

In [84]:
def add_noise_and_compare(X: 'Matrix', noise_level: float = 0.1):
    """
    Вход:
        X: матрица данных (n×m)
        noise_level: уровень шума (доля от значения).
    Выход:
        Сравнение результатов PCA до и после добавления шума.
    """
    # Шаг 1: Добавление шума к данным
    X_noisy = Matrix(X.n, X.m)
    for i in range(X.n):
        for j in range(X.m):
            value = X.get_element(i + 1, j + 1)
            noise = random.gauss(0, noise_level * abs(value))  # Гауссовский шум
            X_noisy.add_value(i + 1, j + 1, value + noise)

    # Шаг 2: PCA до и после добавления шума
    k = min(X.n, X.m)  # Число главных компонент
    X_proj_before, ratio_before = pca(X, k)
    X_proj_after, ratio_after = pca(X_noisy, k)

    # Шаг 3: Вычисление MSE между проекциями
    mse_proj = 0.0
    total = X_proj_before.n * X_proj_before.m
    for i in range(X_proj_before.n):
        for j in range(X_proj_before.m):
            diff = X_proj_before.get_element(i + 1, j + 1) - X_proj_after.get_element(i + 1, j + 1)
            mse_proj += diff ** 2
    mse_proj /= total

    # Вывод результатов
    print("Результаты PCA:")
    print(f"Доля объяснённой дисперсии до шума: {ratio_before:.4f}")
    print(f"Доля объяснённой дисперсии после шума: {ratio_after:.4f}")
    print(f"MSE между проекциями: {mse_proj:.4f}")

    return {
        "explained_variance_ratio_before": ratio_before,
        "explained_variance_ratio_after": ratio_after,
        "mse_proj": mse_proj,
    }

In [71]:
import csv

def load_csv(dataset_name: str) -> list:
    """
    Загрузка датасета.

    Вход:
        dataset_name: название датасета

    Выход:
        Список списков с данными из CSV-файла
    """
    data = []
    try:
        with open(dataset_name, "r", newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            data = list(reader)
    except FileNotFoundError:
        print(f"Файл {dataset_name} не найден.")
    except Exception as e:
        print(f"Произошла ошибка: {e}")


    # print(data[1])
    X = Matrix(len(data) - 1, len(data[1]))
    # print(len(data[1]))

    for i in range(1, len(data)):
      # print(data[i])
      for j in range(len(data[i])):
        # print(data[i][j])
        if j != len(data[1])-1:
          if ".." in data[i][j]:
            # print(data[i])
            X.add_value(i, j + 1, float(data[i][j][:2]))
          else:
            X.add_value(i, j + 1, float(data[i][j]))
        else:
          not_procent = data[i][j]
          if not_procent[-1] == "%":
            not_procent = not_procent[:-4]

          if ",," in not_procent:
            # print(data[i])
            not_procent = not_procent[:-4]

          X.add_value(i, j + 1, float(not_procent))

    return X


# Пример использования
data = load_csv("gameandgrade_new.csv")
# data.print_matrix()

In [117]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def evaluate_knn(X, y, k_neighbors=3):
    """
    Обучение и оценка модели KNN.

    Вход:
        X: матрица признаков
        y: вектор целевых меток
        k_neighbors: количество соседей для KNN

    Выход:
        точность модели KNN
    """
    # Разделение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Обучение KNN
    knn = KNeighborsClassifier(n_neighbors=k_neighbors)
    knn.fit(X_train, y_train)

    # Предсказания
    y_pred = knn.predict(X_test)

    # Оценка точности
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


def separate_features_target(data_matrix: Matrix) -> Tuple[Matrix, Matrix]:
    """
    Разделяет матрицу данных на признаки и целевую переменную.

    """
    X_data = []
    y_data = []

    # print(data_matrix.rows)

    for row in data_matrix.rows:
        # print(data_matrix.n[i])

        # Проверка количества элементов в строке
        if len(row) < 3:
            raise ValueError(f"Строка {row} содержит недостаточно данных ({len(row)} элементов)")

        # Признаки: первые два столбца
        X_data.append([row[0], row[1]])

        # Целевая переменная: третий столбец
        y_data.append([row[2]])

    return X_data, y_data


def apply_pca_to_dataset(dataset_name: str, k: int) -> Tuple['Matrix', float]:
    """
    Применение PCA к датасету и оценка качества модели KNN.

    Вход:
        dataset_name: название датасета
        k: число главных компонент

    Выход:
        кортеж (проекция данных, качество модели)
    """
    # Загрузка данных
    data = load_csv(dataset_name)

    # Разделение на признаки и целевые метки
    X_data, y_data = separate_features_target(data)

    # Применение PCA
    X_proj, _ = pca(data, k)

    X = Matrix(len(X_data), len(X_data[0]))
    for i in range(len(X_data)):
        for j in range(len(X_data[i])):
            X.add_value(i + 1, j + 1, float(X_data[i][j]))

    # Оценка качества модели KNN на исходных данных
    original_accuracy = evaluate_knn(X.rows, y_data)

    # Оценка качества модели KNN на проекциях PCA
    pca_accuracy = evaluate_knn(X_proj.rows, y_data)

    return X_proj, pca_accuracy

In [118]:
X_proj, pca_accuracy = apply_pca_to_dataset("gameandgrade_new.csv", k=2)

print("Проекция данных:")
X_proj.print_matrix()
print("Точность модели KNN после PCA:", pca_accuracy)

Центрированная матрица
[-0.4987012987012987, -3.944155844155844, -0.5844155844155845, -0.24285714285714288, -0.48831168831168825, 0.2935064935064935, 2.1623376623376624, 0.28181818181818175, 1.5896103896103897, -0.018870129870123264, 9.9415584415583]
[0.5012987012987014, -3.944155844155844, -0.5844155844155845, 0.7571428571428571, -0.48831168831168825, 0.2935064935064935, -0.8376623376623376, -0.7181818181818183, -0.4103896103896103, 5.481129870129877, 559.9415584415583]
[-0.4987012987012987, -3.944155844155844, -1.5844155844155845, -2.242857142857143, -1.4883116883116883, -0.7064935064935065, -0.8376623376623376, -0.7181818181818183, -0.4103896103896103, 2.4811298701298767, 259.9415584415583]
[-0.4987012987012987, -3.944155844155844, 1.4155844155844155, 2.757142857142857, -0.48831168831168825, 0.2935064935064935, 0.16233766233766245, -1.7181818181818183, -0.4103896103896103, -32.51887012987012, -3240.0584415584417]
[0.5012987012987014, -3.944155844155844, -0.5844155844155845, -1.24285

/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [124]:
# Визуализация проекции
fig = plot_pca_projection(X_proj)

X_proj.print_matrix()
# Отображение графика
plt.show(fig)
fig.savefig("test_x_proj_on_kaggle.png", dpi=300)

[-9.939633779378296, 0.7158249995169806]
[-559.9663749292088, 0.7242273151411123]
[-259.9517217996677, 0.4157871090029053]
[3240.2231745587114, 1.8015090379454364]
[-759.9764551544107, 0.5591381685758314]
[-259.9510153024686, 0.9820461314745164]
[2240.1731265917083, 1.2909842267879554]
[-259.9513239349611, 0.9981831368357472]
[1740.1477874717307, 1.0788513583929884]
[-1059.9911229488587, 0.865885083289422]
[-259.9511369808286, 0.8204793860673476]
[3240.2220829446137, 1.09932530593548]
[-1260.0018274039473, 0.6458321131300799]
[340.0779034041606, 0.8814666672594598]
[-1760.0258978580011, 0.9196699970313222]
[2740.198221935332, 1.648783983234427]
[-2060.0408184432385, 0.5855039681750362]
[-1260.0013068200703, 0.6347929852889145]
[-959.9861758702116, 0.8113746230671648]
[2240.1729847901593, 0.9781667319532268]
[-1760.0263062396657, 0.49531893093975654]
[740.0985825621611, 1.0985852505287834]
[740.0986747596176, 1.2122186821295973]
[3240.2230751069474, 1.6003895771074141]
[-259.95194191523